In [ ]:
!pip install -U dataprep

In [ ]:
!pip install autoviz

In [ ]:
!pip install evalml

In [ ]:
pip install "dask[distributed]" --upgrade

In [ ]:
!pip install -U dtale

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()

In [ ]:
import dataprep as dp
from dataprep.eda import create_report, plot, correlation, plot_correlation, plot_missing

In [ ]:
from evalml.preprocessing import split_data
from evalml.automl import AutoMLSearch
from evalml.utils import infer_feature_types

In [ ]:
import dtale
import dtale.app as dtale_app
dtale_app.USE_COLAB = True

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from google.colab import files
import io
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,OrdinalEncoder
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RepeatedStratifiedKFold, RepeatedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, RobustScaler, QuantileTransformer, MinMaxScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, make_scorer, confusion_matrix

In [ ]:
file = files.upload()

In [ ]:
df_train = pd.read_csv(io.BytesIO(file['HepatitisCdata.csv']))
df_train_copy = pd.read_csv(io.BytesIO(file['HepatitisCdata.csv']))

## Train

In [ ]:
df_train

## EDA before Cleaning



*   DataPrep


In [ ]:
df_train.describe()

In [ ]:
report = create_report(df_train)
report

In [ ]:
plot(df_train)

In [ ]:
plot_missing(df_train)

In [ ]:
plot_correlation(df_train)

In [ ]:
# plt.boxplot(df_train['bmi'])



---





*   Autoviz



In [ ]:
af = AV.AutoViz("",dfte=df_train,verbose=1,depVar='Category', lowess=True)



*   DTale



In [ ]:
dtale.show(df_train)



---



---



## Cleaning

In [ ]:
df_train.drop('Unnamed: 0', axis=1, inplace=True)



*   Imputing Missing Value


In [ ]:
def outlier_checking(col):
  Q1 = df_train[col].quantile(0.25)
  Q3 = df_train[col].quantile(0.75)
  IQR = Q3 - Q1
  temp = (df_train[col] < (Q1 - (1.5 * IQR))) | (df_train[col] > (Q3 + (1.5 * IQR)))
  return temp

In [ ]:
for col in df_train.columns:
  if df_train[col].isna().sum() != 0:
    print(col)
    if df_train[col].dtype == object:
      df_train[col].fillna(df_train[col].mode()[0], inplace=True)
    if ((df_train[col].dtype == int) or (df_train[col].dtype == float)):
      if (True in outlier_checking(col)) == True:   # Outlier is present
        outlier.append(col)
        df_train[col].fillna(df_train[col].median(), inplace = True)
      if (True in outlier_checking(col)) == False:  # Outlier is absent
        outlier_absent.append(col)
        df_train[col].fillna(df_train[col].mean(), inplace = True)
  else:
    pass

In [ ]:
bins= [0,2,4,13,20,50,110]
labels = ['Infant','Toddler','Kid','Teen','Adult','Senior']
df_train['AgeGroup'] = pd.cut(df_train['Age'], bins=bins, labels=labels, right=False)
df_train = df_train.astype({"AgeGroup": object})



*   Outlier Identification

*   Categoring Column





In [ ]:
outlier = []
outlier_absent = []
category_column = []
numerical_column = []
for col in df_train.columns:
  if df_train[col].dtype == object:
    category_column.append(str(col))
  if ((df_train[col].dtype == int) or (df_train[col].dtype == float)):
    numerical_column.append(str(col))
    if (True in outlier_checking(col)) == True:   # Outlier is present
      outlier.append(col)
    if (True in outlier_checking(col)) == False:  # Outlier is absent
      outlier_absent.append(col)
print(outlier)
print(outlier_absent)
print(category_column)
print(numerical_column)

## EDA after Cleaning

In [ ]:
df_train.describe()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.info()

In [ ]:
report_after_cleaning = create_report(df_train)
report_after_cleaning

In [ ]:
plot(df_train)

In [ ]:
plot_missing(df_train)

In [ ]:
plot_correlation(df_train)



*   Autoviz



In [ ]:
af_after_cleaning = AV.AutoViz("",dfte=df_train, depVar='Category',verbose=1, lowess=True)



*   DTale



In [ ]:
dtale.show(df_train)



---



## Fit-Transform

In [ ]:
df_train

In [ ]:
RS = RobustScaler()
SSSF = StandardScaler(with_std=False)
SSST = StandardScaler(with_std=True)
QTU = QuantileTransformer(n_quantiles=4, random_state=0, output_distribution='uniform')
QTN = QuantileTransformer(n_quantiles=4, random_state=0, output_distribution='normal')
MMS = MinMaxScaler()
PTSF = PowerTransformer(standardize=False)
PTST = PowerTransformer(standardize=True)

for col in outlier:
  #df_train[[col]] = RS.fit_transform(df_train[[col]])
  df_train[[col]] = QTN.fit_transform(df_train[[col]])
  #df_train[[col]] = QTU.fit_transform(df_train[[col]]) 
  df_train[[col]] = PTST.fit_transform(df_train[[col]])
  df_train[[col]] = SSST.fit_transform(df_train[[col]])
for col in outlier_absent:
  df_train[[col]] = PTST.fit_transform(df_train[[col]])
  df_train[[col]] = SSST.fit_transform(df_train[[col]])

In [ ]:
create_report(df_train)

In [ ]:
create_report(df_train)


## Encoding




*   Label Encoder



In [ ]:
le = LabelEncoder()
for col in category_column:
  df_train[col] = le.fit_transform(df_train[col])

In [ ]:
df_train['Category'].unique()

## Labels

In [ ]:
labels_train = df_train.pop('Category')
labels_train

## Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_train, labels_train, test_size=0.163)
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
x_test.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

In [ ]:
x_train

## All Classifier Testing


*   Different Classifier Testing

---



---







*   Classifier



In [ ]:
KNC = KNeighborsClassifier()
XGB = xgb.XGBClassifier(objective='multi:softmax', learning_rate=0.4, n_estimators=100, max_depth=3)
BNB = BernoulliNB()
GNB = GaussianNB()
DTC = DecisionTreeClassifier()
ETC = ExtraTreeClassifier()
RFC = RandomForestClassifier()
ETsC = ExtraTreesClassifier()
GBC = GradientBoostingClassifier()


estimator = [('kneighbor', KNC),
             ('xgb', XGB),
             ('bernoulli', BNB),
             ('gaussian', GNB),
             ('decisiontree', DTC),
             ('extratree', ETC),
             ('randomforest', RFC),
             ('extratrees', ETsC),
             ('gradientboosting', GBC),
             ]


classifier = [KNeighborsClassifier(), 
              xgb.XGBClassifier(objective='multi:softmax', learning_rate=0.4, n_estimators=100, max_depth=3),
              RidgeClassifier(), 
              SGDClassifier(), 
              PassiveAggressiveClassifier(),
              BernoulliNB(), 
              GaussianNB(),
              DecisionTreeClassifier(), 
              ExtraTreeClassifier(),
              RandomForestClassifier(),
              LinearSVC(max_iter=5000),
              SVC(),
              ExtraTreesClassifier(), 
              GradientBoostingClassifier()
              ]
ensemble = [
            VotingClassifier(estimators=estimator, voting='hard'),
            VotingClassifier(estimators=estimator, voting='soft'),
            AdaBoostClassifier(n_estimators=100),
            BaggingClassifier(n_estimators=100)           
]



---





*   Ensemble


In [ ]:
fig , ax = plt.subplots(nrows = len(ensemble), ncols=1, figsize=(16,40))
for k in range(0,len(ensemble)):
  model_ensemble = ensemble[k]
  model_ensemble = model_ensemble.fit(x_train,y_train)
  kfold_ensemble = StratifiedKFold(n_splits=10)
  results_kfold_ensemble =cross_val_score(model_ensemble, x_train, y_train, cv=kfold_ensemble)
  print("Accuracy: %.2f%%" % (results_kfold_ensemble.mean()*100.0))
  prediction_ensemble = model_ensemble.predict(x_test)
  confusion_matrix_ensemble = pd.crosstab(y_test, prediction_ensemble, rownames=['Actual'], colnames=['Predicted'])
  sb.heatmap(confusion_matrix_ensemble, annot=True, cmap='PuBu', ax=ax[k])
  ax[k].set_title(model_ensemble)
  plt.tight_layout()
  print("{} Done".format(ensemble[k]))
  print('\n')



*   Normal checking



In [ ]:
fig , axes = plt.subplots(nrows = len(classifier), ncols=1, figsize=(16,60))
for i,est in enumerate(classifier):
  print(est)
  model_normal = est
  model_normal = model_normal.fit(x_train, y_train)
  kfold_normal = StratifiedKFold(n_splits=10)
  results_kfold_normal =cross_val_score(model_normal, x_train, y_train, cv=kfold_normal)
  print("Accuracy: %.2f%%" % (results_kfold_normal.mean()*100.0))
  prediction_normal = model_normal.predict(x_test)
  confusion_matrix_normal = pd.crosstab(y_test, prediction_normal, rownames=['Actual'], colnames=['Predicted'])
  sb.heatmap(confusion_matrix_normal, annot=True, cmap='PuBu', ax=axes[i])
  axes[i].set_title(model_normal)
  plt.tight_layout()
  print('\n')



---



---



## Actual Classifier

In [ ]:
model = xgb.XGBClassifier(objective='multi:softmax', learning_rate=0.2, n_estimators=100, max_depth=3)



*   Fit



In [ ]:
model = model.fit(x_train, y_train)



*   K-Fold CV



In [ ]:
kfold1 = StratifiedKFold(n_splits=10)
results_kfold =cross_val_score(model, x_train, y_train, cv=kfold1)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0))



*   Predict



In [ ]:
f2 = model.predict(x_test)

In [ ]:
final = pd.DataFrame(columns=['category_actual','category_predicted'])
final['category_actual'] = y_test
final['category_predicted'] = f2
final.reset_index(inplace=True)
confusion_matrix = pd.crosstab(final['category_actual'], final['category_predicted'], rownames=['Actual'], colnames=['Predicted'])
sb.heatmap(confusion_matrix, annot=True, cmap='PuBu')